In [63]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide

# data preprocess
def data_preprocess(stock_number, threshold = 0):
    stock_df = pd.read_csv(f'C:/Users/111030/Desktop/LSTM-stock-predicting-and-construct-investment-strategy-main/data/{stock_number} 2023.csv', encoding='utf-8', encoding_errors= 'ignore')
    five_day_min_low = talib.MIN(stock_df['Low'], timeperiod=5) # 五天最低價當作long strategy的停損點
    five_day_max_high = talib.MAX(stock_df['High'], timeperiod=5) # 五天最高價當作short strategy的停損點
    stock_df['five_day_min_low'] = five_day_min_low
    stock_df['five_day_max_high'] = five_day_max_high
    stock_df = stock_df[25:-1]
    predicted_price_df = pd.read_csv(f'C:/Users/111030/Desktop/LSTM-stock-predicting-and-construct-investment-strategy-main/predicted_result/ResGRU {stock_number} 2023 predicted prices.csv', encoding='utf-8', encoding_errors = 'ignore')
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])
    stock_df.index = stock_df['Date']
    stock_df['predicted_price'] = predicted_price_df['predicted_price'].values
    stock_df['true_price'] = predicted_price_df['true_price'].values

    # long short condition
    stock_df['predicted_price_up'] = (stock_df['predicted_price'] > stock_df['predicted_price'].shift(1)).astype(int)
    stock_df['actual_price_up'] = (stock_df['Close'] > stock_df['Close'].shift(1)).astype(int)
    stock_df['long_condition'] = (stock_df['true_price'].shift(1)<=stock_df['predicted_price']) & ((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1) >= threshold)
    stock_df['short_condition'] = (stock_df['true_price'].shift(1)>stock_df['predicted_price']) & (abs((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1)) >= threshold)
    stock_df['long_condition'] = stock_df['long_condition'].astype(int)
    stock_df['short_condition'] = stock_df['short_condition'].astype(int)
    
    return stock_df

# create signals
def create_signals(stock_df):
    signals_long = pd.Series(np.zeros(len(stock_df)), index=stock_df.index)
    partition_size_long = 0
    stop_loss_long = 0
    for i in range(len(stock_df)):
        if partition_size_long == 0:
            if stock_df['long_condition'][i] == 1:
                signals_long[i] = 1
                partition_size_long = 1
                stop_loss_long = stock_df['five_day_min_low'][i]
        if partition_size_long == 1:
            if (stock_df['Close'][i] <= stop_loss_long) or (stock_df['short_condition'][i] == 1): # 停利停損
                signals_long[i] = -1
                stop_loss_long = 0 # 停損點重置
                partition_size_long = 0 # 平倉

    # short strategy
    signals_short = pd.Series(np.zeros(len(stock_df)), index=stock_df.index)
    partition_size_short = 0
    stop_loss_short = 0
    for i in range(len(stock_df)):
        if partition_size_short == 0:
            if stock_df['short_condition'][i] == 1:
                signals_short[i] = -1
                partition_size_short = -1
                stop_loss_short = stock_df['five_day_max_high'][i]
        if partition_size_short == -1:
            if (stock_df['Close'][i] >= stop_loss_short) or (stock_df['long_condition'][i] == 1):# 停利停損
                signals_short[i] = 1
                stop_loss_short = 0 # 停損點重置
                partition_size_short = 0 # 平倉

    entries_long = signals_long == 1
    exits_long = signals_long == -1
    entries_short = signals_short == -1
    exits_short = signals_short == 1
    
    return entries_long, exits_long, entries_short, exits_short
    
    

In [77]:
data = pd.read_csv(r'C:\Users\111030\Desktop\select_stock\selected_stock.csv')
selected_stock = data['stock_number'].to_numpy()
entries_long_dfs = []
exits_long_dfs = []
entries_short_dfs = []
exits_short_dfs = []
price_dfs = []

for stock in selected_stock:
    stock_df = data_preprocess(stock)
    entries_long, exits_long, entries_short, exits_short = create_signals(stock_df)
    price_dfs.append(pd.DataFrame({f'{stock}': stock_df['Open']}, index=stock_df.index))
    entries_long_dfs.append(pd.DataFrame({f'{stock}': entries_long}, index=stock_df.index))
    exits_long_dfs.append(pd.DataFrame({f'{stock}': exits_long}, index=stock_df.index))
    entries_short_dfs.append(pd.DataFrame({f'{stock}': entries_short}, index=stock_df.index))
    exits_short_dfs.append(pd.DataFrame({f'{stock}': exits_short}, index=stock_df.index))

all_prices = pd.concat(price_dfs, axis=1, join='outer')
all_prices.fillna(all_prices.median(), inplace=True)  
all_entries_long = pd.concat(entries_long_dfs, axis=1, join='outer')
all_entries_long.fillna(False, inplace=True)
all_exits_long = pd.concat(exits_long_dfs, axis=1, join='outer')
all_exits_long.fillna(False, inplace=True)
all_entries_short = pd.concat(entries_short_dfs, axis=1, join='outer')
all_entries_short.fillna(False, inplace=True)
all_exits_short = pd.concat(exits_short_dfs, axis=1, join='outer')
all_exits_short.fillna(False, inplace=True)

all_prices = all_prices.astype(float)
all_entries_long = all_entries_long.astype(bool)
all_exits_long = all_exits_long.astype(bool)
all_entries_short = all_entries_short.astype(bool)
all_exits_short = all_exits_short.astype(bool)

pf = vbt.Portfolio.from_signals(all_prices, 
                                    entries=all_entries_long,
                                    exits=all_exits_long, 
                                    short_entries=all_entries_short,
                                    short_exits=all_exits_short ,
                                    fees = 0,
                                    freq='1D',
                                    direction='both',)
# print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
print(pf.stats().to_string())


C:\Users\111030\AppData\Local\Temp\ipykernel_34000\252630917.py:39: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\111030\AppData\Local\Temp\ipykernel_34000\252630917.py:40: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`

C:\Users\111030\AppData\Local\Temp\ipykernel_34000\252630917.py:42: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\111030\AppData\Local\Temp\ipykernel_34000\252630917.py:44: FutureWarning:

Serie

Start                               2023-02-17 00:00:00
End                                 2023-12-28 00:00:00
Period                                213 days 00:00:00
Start Value                                       100.0
End Value                                    280.459086
Total Return [%]                             180.459086
Benchmark Return [%]                          42.875809
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              15.627555
Max Drawdown Duration                  44 days 14:24:00
Total Trades                                       47.3
Total Closed Trades                                46.5
Total Open Trades                                   0.8
Open Trade PnL                                21.554207
Win Rate [%]                                  66.468681
Best Trade [%]                                35.112549
Worst Trade [%]                               -8

C:\Users\111030\AppData\Local\Temp\ipykernel_34000\1021168981.py:25: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\111030\AppData\Local\Temp\ipykernel_34000\1021168981.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\111030\AppData\Local\Temp\ipykernel_34000\1021168981.py:35: UserWarning:

direction has no effect if short_entries and short_exits are set

C:\Users\111030\AppData\Local\Temp\ipykernel_34000\1021168981.py:44: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x000002BDFE42BD30>. Pass column 